<a href="https://colab.research.google.com/github/YohanWloczysiak/Projet_Python_A2_S1/blob/main/Manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ce notebook permet de récupérer et de traiter les données que nous utiliserons
dans le projet.


# **Préliminaires**


In [ ]:
import urllib
import bs4
import pandas
import re
import numpy

# **Données sur le trafic aérien en France**


Les données sur le trafic aérien en France sont issues du site https://www.ou-et-quand.net/vols/. Afin de les récupérer, nous allons de voir faire du webscraping.

Nous commençons par récupérer l'adresse url de chaque page qui nous intéresse, c'est-à-dire chaque page concernant un aéroport français.

In [ ]:
# (A modifier : voir si on peut récupérer les noms de villes avec du webscraping)

url_radical = "https://www.ou-et-quand.net/vols/"
ordre_villes = ['Strasbourg', 'Bordeaux', 'Clermont-Ferrand', 'Rennes', 'Brest', 'Ajaccio', 'Bastia', 'Calvi', 'Figari', 'Montpellier', 'Limoges', 'Metz', 'Toulouse', 'Lille', 'Caen', 'Nantes', 'Poitiers', 'Marseille', 'Nice', 'Lyon', 'Paris']
url_terminaison = ["alsace/strasbourg/", "aquitaine/bordeaux/", "auvergne/clermont-ferrand/", "bretagne/rennes/", "bretagne/brest/", "corse/ajaccio/", "corse/bastia/", "corse/calvi/", "corse/figari/", "languedoc-roussillon/montpellier/", "limousin/limoges/", "lorraine/metz/", "midi-pyrenees/toulouse/", "nord-pas-de-calais/lille/", "normandie/caen/", "pays-de-la-loire/nantes/", "poitou-charentes/poitiers/", "provence-alpes-c-te-d-azur/marseille/", "provence-alpes-c-te-d-azur/nice/", "rh-ne-alpes/lyon/", "ile-de-france/paris/"]

Pour chaque aéroport, nous devons récupérer plusieurs données : le prix moyen du billet aller-retour en fonction des aéroports de départ, le prix moyen du billet aller-retour en fonction du mois de départ et de l'aéroport de départ ainsi que le prix moyen du billet aller-retour en fonction du délai entre la réservation et le départ et de l'aéroport de départ.

### **Construction des expressions régulières**

On commence par construire les expressions régulières qui vont nous permettre de sélectionner les parties de la page html qui nous intéressent.

Pour récupérer les données, il faudra utiliser plusieurs expressions régulières à chaque fois.






Les expressions commençant par "expression_villes" permettent de récupérer les noms des villes de départ.

In [ ]:
expression_villes = re.compile("categories:.+],")
expression_villes_bis = re.compile(":.+],")
expression_villes_ter = re.compile("[\w]+")

Les expressions commençant par "expression_prix" permettent de récupérer les prix moyens en fonction de l'aéroport de départ.

In [ ]:
expression_prix = re.compile("""'aéroport de départ',
            data: .+,""")
expression_prix_bis = re.compile(":.+],")
expression_prix_ter = re.compile("[\d]+")

Les expressions commençant par "expression_mois" permettent de récupérer la liste des mois pour lesquels on a des données de prix.

In [ ]:
expression_mois = re.compile("""xAxis: {
            categories: .+,
            title: {
                text: null""")
expression_mois_select = re.compile("'jan.'|'fév.'|'mar.'|'avr.'|'mai'|'juin'|'juil'|'août'|'sep.'|'oct.'|'nov.'|'déc.'")
expression_mois_bis = re.compile("[\w]+")

Les expressions commençant par "expression_prix_mois" permettent de récupérer les prix moyens en fonction du mois de départ et de l'aéroport de départ.

In [ ]:
expression_prix_mois = re.compile("""en euros selon le mois de départ',
            data: .+,""")
expression_prix_mois_bis = re.compile(":.+],")
expression_prix_mois_ter = re.compile("[\d]+")

Les expressions commençant par "expression_avance" permettent de récupérer les délais entre la réservation et le départ pour lesquels on a des données de prix.

In [ ]:
expression_avance = re.compile("""xAxis: {
            categories: .+,
            title: {
                text: null""")
expression_avance_select = re.compile("'0 à 1 mois'|'1 à 2 mois'|'2 à 3 mois'|'3 à 4 mois'|'4 à 5 mois'|'5 à 6 mois'|'6 à 7 mois'|'7 à 8 mois'|'8 à 9 mois'|'9 à 10 mois'|'10 à 11 mois'|'11 à 12 mois'")
expression_avance_bis = re.compile("[\w]+")

Les expressions commençant par "expression_prix_avance" permettent de récupérer les prix moyens en fonction du délai entre le mois de départ et la réservation.

In [ ]:
expression_prix_avance = re.compile("""selon le nombre de mois entre la réservation et le vol',
            data: .+,""")
expression_prix_avance_bis = re.compile(":.+],")
expression_prix_avance_ter = re.compile("[\d]+")

### **Récupération des données par webscraping**

Maintenant qu'on a nos expressions régulières, on peut rechercher l'ensemble des données qui nous intéressent sur les pages du site.

In [ ]:
prix_moyen_destination = []
villesbis = []
villes = []
prix = []
liste_prix = []
prix_mois = []
liste_prix_mois = []
prix_avance = []
liste_prix_avance = []
avance_mois = []
mois = []
liste_mois = []


for i in range(len(ordre_villes)):
    url_totale = url_radical+url_terminaison[i]
    
    from urllib.request import Request, urlopen
    req = Request(url_totale, headers={'User-Agent': 'Mozilla/5.0'})
    web_byte = urlopen(req).read()
    page = bs4.BeautifulSoup(web_byte, 'lxml')
    
    prix_moyen_destination.append(page.findAll("script")[23])
    s = prix_moyen_destination[i].text.strip()
    
    # seconde étape : recherche
    # Récupération des villes de départ
    city = expression_villes.findall(s)[0]
    city_bis = expression_villes_bis.findall(city)
    liste_villes = expression_villes_ter.findall(city_bis[0])
    villesbis.append(liste_villes)
    
    
    """Récupération des prix moyens en fonction de la ville de départ"""
    price_moyen_bis = expression_prix.findall(s)
    price_moyen = expression_prix_bis.findall(price_moyen_bis[0])
    prix_moyen = expression_prix_ter.findall(price_moyen[0])
    
    liste_prixbis = []

    for elem in prix_moyen:
        elem_bis = int(elem)
        liste_prixbis.append(elem_bis)
        
    liste_prix.append(liste_prixbis)
    
    
    """Récupération des prix moyens en fonction du mois de départ"""
    price_mois_bis = expression_prix_mois.findall(s)
    month = expression_mois.findall(s)
    for elem in month:
        if expression_mois_select.findall(elem) != []:
            month_select = expression_mois_select.findall(elem)
            
            for elem_bis in month_select:
                month_bis = expression_mois_bis.findall(elem_bis)
                mois.append(month_bis[0])
        
        
    for j in range(len(price_mois_bis)):
        price_mois = expression_prix_mois_bis.findall(price_mois_bis[j])
        prix_mois = expression_prix_mois_ter.findall(price_mois[0])
        liste_prixbis_mois = []

        for elem in prix_mois:
            elem_bis = int(elem)
            liste_prixbis_mois.append(elem_bis)

        liste_prix_mois.append(liste_prixbis_mois) 
    

    
    """Récupération des prix moyens en fonction du mois de réservation"""
    price_avance_bis = expression_prix_avance.findall(s)
    avance = expression_avance.findall(s)
    for elem in avance:
        if expression_avance_select.findall(elem) != []:
            avance_select = expression_avance_select.findall(elem)
            
            for elem_bis in avance_select:
                avance_bis = expression_avance_bis.findall(elem_bis)
                avance_mois.append(avance_bis[0] + ' ' + avance_bis[1] + ' ' +  avance_bis[2] + ' ' +  avance_bis[3])
        
        
    for j in range(len(price_avance_bis)):
        price_avance = expression_prix_avance_bis.findall(price_avance_bis[j])
        prix_avance = expression_prix_avance_ter.findall(price_avance[0])
        liste_prixbis_avance = []

        for elem in prix_avance:
            elem_bis = int(elem)
            liste_prixbis_avance.append(elem_bis)

        liste_prix_avance.append(liste_prixbis_avance)

### **Création des bases de données finales**

La première base de données contiendra les prix moyens du billet aller-retour en fonction uniquement de l'aéroport de départ.

In [ ]:
col_1_1 = []
col_1_2 = []
col_1_3 = []
col_1_4 = []

for l in range(len(villesbis)):
    col_1_1 = col_1_1 + villesbis[l]
    col_1_2 = col_1_2 + [ordre_villes[l]]*len(villesbis[l])
    col_1_3 = col_1_3 + liste_prix[l]

In [ ]:
table_prix_trajet = pandas.DataFrame({'Départ' : col_1_1 , 'Arrivée': col_1_2 , 'Prix': col_1_3})
table_prix_trajet

,Départ,Arrivée,Prix
0,Paris,Strasbourg,248
1,Nantes,Strasbourg,177
2,Marseille,Strasbourg,215
3,Bordeaux,Strasbourg,227
4,Bruxelles,Strasbourg,205
...,...,...,...
162,Bruxelles,Paris,254
163,Genève,Paris,184
164,Lyon,Paris,170
165,Toulouse,Paris,122


La deuxième base de données contiendra les prix moyens du billet aller-retour en fonction du mois de départ et de l'aéroport de départ.

In [ ]:
col_2_1 = []
col_2_2 = []
col_2_3 = []
col_2_4 = mois
num_list = 0

for k in range(len(ordre_villes)):
    for l in range(len(villesbis[k])):
        col_2_1 = col_2_1 + [villesbis[k][l]]*len(liste_prix_mois[num_list])
        col_2_2 = col_2_2 + [ordre_villes[k]]*len(liste_prix_mois[num_list])
        col_2_3 = col_2_3 + liste_prix_mois[num_list]
        num_list = num_list + 1

In [ ]:
table_prix_mois = pandas.DataFrame({'Départ' : col_2_1 , 'Arrivée': col_2_2 , 'Prix': col_2_3, 'Mois': col_2_4})
table_prix_mois[:100]

,Départ,Arrivée,Prix,Mois
0,Paris,Strasbourg,310,jan
1,Paris,Strasbourg,189,fév
2,Paris,Strasbourg,225,mar
3,Paris,Strasbourg,234,avr
4,Paris,Strasbourg,256,mai
...,...,...,...,...
95,Nice,Strasbourg,130,fév
96,Nice,Strasbourg,186,mar
97,Nice,Strasbourg,154,avr
98,Nice,Strasbourg,164,mai


La troisième base de données contiendra les prix moyens du billet aller-retour en fonction du délai entre la réservation et le départ et de l'aéroport de départ.

In [ ]:
col_3_1 = []
col_3_2 = []
col_3_3 = []
col_3_4 = avance_mois
num_list = 0

for k in range(len(ordre_villes)):
    for l in range(len(villesbis[k])):
        col_3_1 = col_3_1 + [villesbis[k][l]]*len(liste_prix_avance[num_list])
        col_3_2 = col_3_2 + [ordre_villes[k]]*len(liste_prix_avance[num_list])
        col_3_3 = col_3_3 + liste_prix_avance[num_list]
        num_list = num_list + 1

In [ ]:
table_prix_avance = pandas.DataFrame({'Départ' : col_3_1 , 'Arrivée': col_3_2 , 'Prix': col_3_3, 'Temps entre la réservation et le vol': col_3_4})
table_prix_avance[:100]

,Départ,Arrivée,Prix,Temps entre la réservation et le vol
0,Paris,Strasbourg,315,0 à 1 mois
1,Paris,Strasbourg,237,1 à 2 mois
2,Paris,Strasbourg,198,2 à 3 mois
3,Paris,Strasbourg,201,3 à 4 mois
4,Paris,Strasbourg,208,4 à 5 mois
...,...,...,...,...
95,Paris,Bordeaux,134,2 à 3 mois
96,Paris,Bordeaux,155,3 à 4 mois
97,Paris,Bordeaux,181,4 à 5 mois
98,Paris,Bordeaux,163,5 à 6 mois


On termine en créant des fichiers csv contenant les données récupérées.

In [ ]:
table_prix_trajet.to_csv('bdd1.csv')
table_prix_mois.to_csv('bdd2.csv')
table_prix_avance.to_csv('bdd3.csv')

# **Données sur le trafic ferroviaire en France**


Les données sur le trafic ferroviaire en France proviennent du bases de données fournies par la SNCF. On commence donc par ouvrir cette base de données.

In [ ]:
df = pandas.read_csv('tarifs-tgv-par-od.csv')
df.columns =['Colonne']
df

FileNotFoundError: ignored

On observe que la base de données récupérée possède une seule colonne et 4162 lignes. Chaque ligne correspond à un trajet (gare de départ-gare d'arrivée), tarif d'appel en seconde classe, plein tarif de loisir en seconde classe et tarif de première classe. Il faut donc créer un DataFrame contenant ces informations séparées en 5 colonnes.

### **Création de deux colonnes : gare de départ et gare d'arrivée**

In [ ]:
liste_gare_0= []
for k in range(len(df['Colonne'])):
    liste_gare_0.append(re.split(';', df['Colonne'][k])[0])
liste_gare_0

liste_gare_0 contient tous les noms de trajets, que nous allons séparer en deux listes disjointes : celle contenant les fares de départ, et celle contenant les gares d'arrivée. 

In [ ]:
liste_gare_départ=[]
liste_gare_arrivée=[]

for k in range(len(liste_gare_0)):
    liste_gare_départ.append(re.split('-', liste_gare_0[k])[0])
    liste_gare_arrivée.append(re.split('-', liste_gare_0[k])[1])

### **Création des colonnes tarifs**

In [ ]:
tarifs = []
for k in range(len(df['Colonne'])) :
    tarifs += re.split(';', df['Colonne'][k])
tarifs

prix_dappel_2nde = tarifs[1::5]
plein_tarif_loisir_2nde = tarifs[2::5]
premiere_classe = tarifs[3::5]

#print(len(prix_dappel_2nde), len(plein_tarif_loisir_2nde), len(premiere_classe))

In [ ]:
bdd_sncf = pd.DataFrame({'Gare de départ' :liste_gare_départ, 'Gare d arrivée': liste_gare_arrivée, 'Prix d appel seconde':prix_dappel_2nde, 'Plein tarif loisir seconde':plein_tarif_loisir_2nde,'Tarif premiere classe' :premiere_classe})
bdd_sncf

### **Ajout des distances entre deux gares** 

On récupère une nouvelle base de données contenant pour chaque gare ses coordonnées GPS (latitude et longitude). A partir de ces données, on va déterminer la distance pour l'ensemble des trajets contenus dans bdd_sncf. 

In [ ]:
coordonnees = pd.read_csv('gares-tgv.csv', encoding = 'latin-1')
coordonnees = coordonnees.drop(['Unnamed: 0', 'INSEE_REG', 'NOM_REG','INSEE_DEP',
       'NOM_DEP'], axis=1)
coordonnees

On représente l'ensemble des gares sur une carte de la France. 

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

points = [Point(lon, lat) for ind, lat, lon in coordonnees[['Latitude', 'Longitude']].itertuples()]
enedis = gpd.GeoDataFrame(data=dict(geometry=points))
enedis['Nom'] = coordonnees['Nom_Gare']
enedis.head()

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head()

from shapely.geometry import Polygon
europe = world[world.continent == "Europe"].copy()
europe['geometry'] = europe.geometry.intersection(Polygon([(-10,35), (50,35), (50,70), (-10, 70)]))
europe.head()

fig, ax = plt.subplots(1, 1, figsize=(5, 4))
minx, miny, maxx, maxy = enedis.total_bounds

europe.plot(ax=ax, color='white', edgecolor='black')
enedis.plot(ax=ax, marker='o', color='red', markersize=2)
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)
ax.set_title("Localisation des gares d'après la base de données récupérée");

On utilise le module Geod de pyproj pour déterminer les distances en kms. 

In [ ]:
from pyproj import Geod

In [ ]:
wgs84_geod = Geod(ellps ='WGS84')


def Distance(lat1,lon1,lat2,lon2):
     az12,az21,dist = wgs84_geod.inv(lon1,lat1,lon2,lat2) 
     return dist

In [ ]:
nom_gare = coordonnees['Nom_Gare'].tolist()
latitude = coordonnees['Latitude'].tolist()
longitude = coordonnees['Longitude'].tolist()

liste = [[] for i in range(len(nom_gare))]

for i in range(len(liste)) : 
    liste[i] = nom_gare[i], latitude[i], longitude[i]
    
#liste

liste contient les triplets (nom de gare, latitude, longitude) pour chaque gare. Pour chaque couple de gares contenues dans cette liste, on calcule la distance entre ces deux gares que l'on stocke dans le dictionnaire distance. 

In [ ]:
distance = {}
for k in range(len(liste)) : 
    for j in range(len(liste)) : 
        distance[liste[j][0], liste[k][0]] = Distance(liste[j][1],liste[j][2], liste[k][1], liste[k][2])
        
        
gares = list(distance.keys())
gare_depart = [gares[k][0] for k in range(len(gares))]
gare_arrivee = [gares[k][1] for k in range(len(gares))]

On crée le DataFrame distance_villes qui pour une gare de départ et une gare d'arrivée donne la distance en kms entre ces deux gares. 

In [ ]:
distance_villes = pd.DataFrame({'Gare de départ' : gare_depart, 'Gare d arrivée' : gare_arrivee, 'Distance entre les villes' : list(distance.values())})
distance_villes['Distance entre les villes'] = distance_villes['Distance entre les villes']/1000

  On veut désormais combiner les deux DataFrame : bdd_sncf et distance_villes, afin d'obtenir un DataFrame final contenant : 
- le nom de la gare de départ, 
- le nom de la gare d'arrivée, 
- les tarifs correspondants,
- la distance correspondante à ce trajet. 

    Cependant, ces deux bases de données étant issues de jeux de données différents, **la syntaxe des noms de gares est différente**. Par exemple, les noms de gares dans bdd_sncf sont affichés en majuscule et sans caractères spéciaux (en particulier les tirets). On doit donc modifier la syntaxe des noms de gares contenues dans distance_villes. 

In [ ]:
distance_villes['Gare de départ'] = distance_villes['Gare de départ'].str.upper()
distance_villes['Gare d arrivée'] = distance_villes['Gare d arrivée'].str.upper()
distance_villes = distance_villes.replace('\-', ' ', regex = True)
distance_villes.iloc[2502]

Il reste néanmoins des différences entre les noms qui pointent vers des gares identiques. Pour résoudre ce problème on utilise le module Levenshtein qui permet de définir un **ratio de ressemblance entre deux chaînes de caractères**. 
On crée la fonction *ratio* qui pour une gare de départ $x$ et une gare d'arrivée $y$ de bdd_sncf, retourne l'indice de la ligne de distance_villes du trajet de $x$ à $y$ correspondant. 

In [ ]:
import Levenshtein as lev

garedepartmaj = distance_villes['Gare de départ'].tolist() 
garearrmaj = distance_villes['Gare d arrivée'].tolist()

def ratio(x,y):
    garedep = []
    ratiosd = []
    ratiosa =[]
    garrearr =[]
    for gare in garedepartmaj :
        Ratio_dep = lev.ratio(gare, x) # on calcule pour chaque gare de départ de distance_villes le ratio de similitude avec x
        ratiosd.append(Ratio_dep) # la liste ratiosd stocke l'ensemble des ratios
    ratiomaxd = max(ratiosd) #on prend le max de ces ratios
    
    for i in range (len(ratiosd)):
        if ratiosd[i] == ratiomaxd:
            garedep.append(i) # on stocke dans garedep les indices des gares de départ pour lesquelles le ratio est maximal
    
    for index in garedep : 
        Ratio_arr = lev.ratio(garearrmaj[index], y) #on regarde les ratios de similitude avec la gare d'arrivée pour les gares dans garedep 
        ratiosa.append(Ratio_arr)
    
    ratiomaxa=max(ratiosa)
    for i in range (len(ratiosa)):
        if ratiosa[i] == ratiomaxa:
            return(garedep[i]) #on retourne l'indice de la ligne de distances_villes pour laquelle la gare de départ et la gare 
        #d'arrivée ont le plus grand ratio de similitude avec les gares x et y. 


In [ ]:
distancegares=[]
for i in range(len(liste_gare_départ)):
    départ =liste_gare_départ[i]
    arrivée = liste_gare_arrivée[i]
    distancegares.append(distance_villes['Distance entre les villes'][ratio(départ,arrivée)])

#print(distancegares) 
# distancegares contient l'ensemble des distances pour les trajets de bdd_sncf. 


On ajoute finalement la colonne distance à la DataFrame bdd_sncf. 

In [ ]:
bdd_sncf['Distance du trajet'] = distancegares
bdd_sncf
#len(distancegareskm)

In [ ]:
bdd_sncf.to_csv('bdd_sncf.csv', header = True, index = False)